In [146]:
%load_ext autoreload
%autoreload 2
from expressiveness_benchmark.types import Plan, Task, Language, SourceRange, Program, save_languages
from code_widget.example import CodeWidget
from dataclasses import replace
import json
import pandas as pd

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [2]:
# CHANGE ME!
TASK_ID = 'row_per_child'
AUTHOR = 'will'

In [115]:
task = Task(
    id=TASK_ID,
    category="Joins",
    name="Row per family to row per child",
    description="TODO",
    plan=[],
    sample_input={
        "families": [
            {"family": 1, "dob_child1": "1998", "dob_child2": "1999", "dob_child3": "2000", "height_child1": "5_11", "height_child2": "5_10",  "height_child3": "5_9"},
            {"family": 2, "dob_child1": "1997", "dob_child2": "1995", "dob_child3": "1996", "height_child1": "5_3", "height_child2": "5_5", "height_child3": "5_1"}
        ]
    },
    sample_output=[
        {"family": 1, "child": "child1", "dob": "1998", "height": "5_11"},
        {"family": 1, "child": "child2", "dob": "1999", "height": "5_10"},
        {"family": 1, "child": "child3", "dob": "2000", "height": "5_9"},
        {"family": 2, "child": "child1", "dob": "1997", "height": "5_3"},
        {"family": 2, "child": "child2", "dob": "1995", "height": "5_5"},
        {"family": 2, "child": "child3", "dob": "1996", "height": "5_1"},
    ]
)
task.save()

prototype = Program(
    task=TASK_ID,
    author=AUTHOR,
    language=''    
)

In [116]:
r = replace(
    prototype,
    language='r',
    source='''
row_per_child <- function(families) {
  families %>%
    pivot_longer(
      !family,
      names_to = c(".value", "child"),
      names_sep = "_",
    )
}''')
r.execute(task)
r.save()

In [122]:
sql = replace(
    prototype,
    language='sql',
    source='''
SELECT 
  family,
  ('child' || child) AS child,
  (CASE child 
    WHEN 1 THEN dob_child1 
    WHEN 2 THEN dob_child2 
    WHEN 3 THEN dob_child3 
   END) AS dob,
  (CASE child 
    WHEN 1 THEN height_child1 
    WHEN 2 THEN height_child2 
    WHEN 3 THEN height_child3 
   END) AS height
FROM 
  families
  CROSS JOIN
  (SELECT 1 as child UNION VALUES (2), (3))
''')
sql.execute(task)
sql.save()

In [17]:
sql.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "sql", "plan": {}, "source": "SELECT DISTINCT…

In [134]:
datalog1 = replace(
    prototype,
    language='datalog',
    source='''
row_per_child("child1", dob, family, height) :-
  families(dob, _, _, family, height, _, _).
row_per_child("child2", dob, family, height) :-
  families(_, dob, _, family, _, height, _).
row_per_child("child3", dob, family, height) :-
  families(_, _, dob, family, _, _, height).
''')
datalog1.execute(task)
datalog1.save()

In [26]:
datalog2.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "datalog", "plan": {}, "source": ".decl diffe…

In [142]:
pandas = replace(
    prototype,
    language='python-pandas',
    author='will',
    source='''def row_per_child(families):
  df = pd.wide_to_long(
    families, 
    stubnames=['dob', 'height'], 
    sep="_child", 
    i='family', 
    j='child').reset_index()
  df.child = df.child.map(lambda c: f'child{c}')
  return df
  ''')
pandas.execute(task)
pandas.save()

In [15]:
pandas.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "python-pandas", "plan": {}, "source": "def u…

In [143]:
python_functional = replace(
    prototype,
    language='python-functional',
    source='''def row_per_child(families):
  return [
    {
      'family': family['family'],
      'child': f'child{i}',
      'dob': family[f'dob_child{i}'],
      'height': family[f'height_child{i}']
    }
    for family in families    
    for i in [1, 2, 3]
  ]
  ''')
python_functional.execute(task)
python_functional.save()

In [11]:
python_functional.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "python-functional", "plan": {}, "source": "d…

In [145]:
python_imperative = replace(
    prototype,
    language='python-imperative',
    source='''
def row_per_child(families):
  children = []
  for family in families:
    for i in [1, 2, 3]:
      children.append({
        'family': family['family'],
        'child': f'child{i}',
        'dob': family[f'dob_child{i}'],
        'height': family[f'height_child{i}']
      })
  return children
''')
python_imperative.execute(task)
python_imperative.save()

In [13]:
python_imperative.widget(task)

Output()

CodeWidget(program='{"task": "unique_beer_drinkers", "language": "python-imperative", "plan": {}, "source": "d…